In [6]:
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re


In [7]:
path_dir = "C:\\Users\\Modern\\Мой диск\\Диссертация\\Data\\Magnetic field data\\Membrane levitating system 2 take"
file_path = "C:\\Users\\Modern\\Мой диск\\Диссертация\\Data\\Magnetic field data\\Membrane levitating system 2 take\\1mm.dat"


for address, dirs, files in os.walk(path_dir):
    for file in files:
        if ".dat" in str(file):
            z = re.search(r'([0-9]+)',file)
            z_coord = z[0]
            print(z_coord)

# field=pd.read_csv(file_path,encoding = 'unicode_escape',delimiter='\t',names= ['B','x','y'],skiprows=[0])
# field_np_1d = field['B'].to_numpy()

# field_np = field_np_1d.reshape(21,21)

# sns.scatterplot(data=field, x="x", y="y", hue = 'B')
# sns.heatmap(data=field_np)
# sns.scatterplot(data=field_np)





10
11
12
13
15
1
2
3
4
5
6
7
8
9
